# Data Ingestion, Cleaning, Merging

In [1]:
#necessary libraries
import chart_studio.plotly as py
import pandas as pd

In [2]:
#reading in BLS Unemployment data
unemployment = pd.read_excel('data/BLS_Unemployment.xlsx', header = 11)
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
unemployment = unemployment.rename(columns = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12})
unemployment = pd.melt(unemployment, id_vars = 'Year', value_vars = numbers)
unemployment = unemployment.rename(columns = {'value': 'unemployment'})
unemployment = unemployment.astype(str)
unemployment['unemployment'] = unemployment['unemployment'].astype(float)
unemployment['Date'] = unemployment['Year'] + '-' + unemployment['variable']
unemployment = unemployment[['unemployment', 'Date']]
unemployment['Date']= pd.to_datetime(unemployment['Date']).dt.to_period('M')
unemployment.head()

,unemployment,Date
0,3.4,1948-01
1,4.3,1949-01
2,6.5,1950-01
3,3.7,1951-01
4,3.2,1952-01


In [3]:
#reading in data on google trends for 'unemployment'
g_unemployment = pd.read_csv('data/GTrends_Unemployment.csv', header = 1)
g_unemployment = g_unemployment.rename(columns = {'unemployment: (United States)': 'g_unemployment', 'Month': 'Date'})
g_unemployment['Date'] = pd.to_datetime(g_unemployment['Date']).dt.to_period('M')
g_unemployment.head()

,Date,g_unemployment
0,2004-01,6
1,2004-02,5
2,2004-03,5
3,2004-04,4
4,2004-05,4


In [4]:
#reading data on google searches for 'cheap gas'
g_gas = pd.read_csv('data/GTrends_gas.csv', header = 1)
g_gas = g_gas.rename(columns = {'gas: (United States)': 'g_gas', 'Month': 'Date'})
g_gas['Date'] = pd.to_datetime(g_gas['Date']).dt.to_period('M')

g_gas.head()

,Date,g_gas
0,2004-01,42
1,2004-02,41
2,2004-03,45
3,2004-04,48
4,2004-05,62


In [5]:
#Reading in google data on searches for 'food stamps'
g_food_stamps = pd.read_csv('data/GTrends_food_stamps.csv', header = 1)
g_food_stamps = g_food_stamps.rename(columns = {'food stamps: (United States)': 'g_food_stamps', 'Month': 'Date'})
g_food_stamps['Date'] = pd.to_datetime(g_food_stamps['Date']).dt.to_period('M')

g_food_stamps.head()

,Date,g_food_stamps
0,2004-01,8
1,2004-02,5
2,2004-03,8
3,2004-04,6
4,2004-05,5


In [6]:
#reading in actual gas prices
gas_price = pd.read_excel('data/gas_prices.xls', sheet_name = 'Data', header = 1)
gas_price = gas_price.iloc[:, 0:2]
gas_price = gas_price.rename(columns = {'EMM_EPM0_PTE_NUS_DPG': 'retail_gas', 'Sourcekey': 'Date'})
gas_price = gas_price.drop([0])
# https://www.interviewqs.com/ddi-code-snippets/extract-month-year-pandas
gas_price['Date'] = pd.to_datetime(gas_price['Date']).dt.to_period('M')
gas_price.head()

,Date,retail_gas
1,1990-08,NaN
2,1990-09,NaN
3,1990-10,NaN
4,1990-11,NaN
5,1990-12,NaN


In [7]:
#University of Michigan consumer sentiment
sentiment = pd.read_csv('data/UM_consumer_sentiment.csv')
sentiment = sentiment.rename(columns = {'DATE': 'Date', 'UMCSENT': 'consumer_sentiment'})
sentiment['Date'] = pd.to_datetime(sentiment['Date']).dt.to_period('M')
sentiment.head()

,Date,consumer_sentiment
0,2052-11,NaN
1,2052-12,NaN
2,2053-01,NaN
3,2053-02,NaN
4,2053-03,NaN


### Food Stamps

In [8]:
#enrollment in food stamps
food_stamps = pd.read_excel('data/SNAPZip69throughCurrent-1/FY89.xls', sheet_name = 'US Summary ', header = 5)
food_stamps = food_stamps.iloc[0:12, 0:2]
food_stamps = food_stamps.rename(columns = {'US Summary': 'Date', 'Unnamed: 1': 'fs_households'})
food_stamps['Date'] = pd.to_datetime(food_stamps['Date'], infer_datetime_format = True).dt.to_period('M')

for i in range(1990,2015):
    docname = str(i)[2:]
    #print(docname)
    filename = 'data/SNAPZip69throughCurrent-1/FY' + str(i)[2:] + '.xls'
    temp = pd.read_excel(filename, sheet_name = 'US Summary ', header = 5)
    temp = temp.iloc[0:12, 0:2]
    temp = temp.rename(columns = {'US Summary': 'Date', 'Unnamed: 1': 'fs_households'})
    temp['Date'] = pd.to_datetime(temp['Date'], infer_datetime_format = True).dt.to_period('M')
    food_stamps = pd.concat([food_stamps, temp], axis = 0)

#format of files changed in 2015
#2020 file is empty
for i in range(2015,2020):
    docname = str(i)[2:]
    #print(docname)
    filename = 'data/SNAPZip69throughCurrent-1/FY' + str(i)[2:] + '.xls'
    temp = pd.read_excel(filename, sheet_name = 'US Summary', header = 7)
    temp = temp.iloc[0:12, 0:2]
    temp = temp.rename(columns = {'US Summary': 'Date', 'Unnamed: 1': 'fs_households'})
    temp['Date'] = pd.to_datetime(temp['Date'], infer_datetime_format = True).dt.to_period('M')
    food_stamps = pd.concat([food_stamps, temp], axis = 0)

# Merging Datasets

In [11]:
#lots of merges
data = pd.merge(unemployment, sentiment, how = 'left', left_on = 'Date', right_on = 'Date')
data = pd.merge(data, g_unemployment, how = 'left')
data = pd.merge(data, g_gas, how = 'left')
data = pd.merge(data, g_food_stamps, how = 'left')
data = pd.merge(data, food_stamps, how = 'left')
data = pd.merge(data, gas_price, how = 'left')
data.to_csv('MergedTS_Feb13.csv')
data = data.sort_values(by = 'Date')
data = data[data['Date'] < '2021-02'] # https://stackoverflow.com/questions/22898824/filtering-pandas-dataframes-on-dates
data['consumer_sentiment'] = data['consumer_sentiment'].astype('float64')
data['retail_gas'] = data['retail_gas'].astype('float64')

data.to_csv('MergedTS_Feb17.csv')

data.tail()

,unemployment,Date,consumer_sentiment,g_unemployment,g_gas,g_food_stamps,fs_households,retail_gas
664,7.8,2020-09,80.4,44.0,70.0,41.0,NaN,2.274
738,6.9,2020-10,81.8,34.0,71.0,48.0,NaN,2.248
812,6.7,2020-11,76.9,29.0,68.0,42.0,NaN,2.200
886,6.7,2020-12,80.7,37.0,70.0,45.0,NaN,2.284
73,6.3,2021-01,NaN,44.0,69.0,52.0,NaN,2.420


In [12]:
#examining data types
data.dtypes

unemployment            float64
Date                  period[M]
consumer_sentiment      float64
g_unemployment          float64
g_gas                   float64
g_food_stamps           float64
fs_households           float64
retail_gas              float64
dtype: object